# Pygame guitar circles
https://www.makeartwithpython.com/blog/video-synthesizer-in-python/

In [ ]:
import pyaudio
import aubio
import numpy as np

import pygame
import random

from threading import Thread

import queue
import time

In [ ]:
p = pyaudio.PyAudio()

CHANNELS = 1
RATE = 44100
buffer_size = 4096 # needed to change this to get undistorted audio


# detect recorder
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')

recorder_name = 'Scarlett 2i2 USB'
for i in range(0, numdevices):
    name = p.get_device_info_by_host_api_device_index(0, i).get('name')
    if name == recorder_name:
        recorder_id = i

In [ ]:
recorder_id

In [ ]:
stream = p.open(
    format=pyaudio.paFloat32,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    # stream_callback=callback,
    input_device_index=recorder_id
)

# stream.start_stream()

In [ ]:
pygame.init()

 # run in window
screenWidth, screenHeight = 800, 800
screen = pygame.display.set_mode((screenWidth, screenHeight))

clock = pygame.time.Clock()

In [ ]:
white = (255, 255, 255)
black = (0, 0, 0)

class Circle(object):
    def __init__(self, x, y, color, size):
        self.x = x
        self.y = y
        self.color = color
        self.size = size

    def shrink(self):
        self.size -= 3

colors = [(229, 244, 227), (93, 169, 233), (0, 63, 145), (255, 255, 255), (109, 50, 109)]
circleList = []

In [ ]:
# setup onset detector
tolerance = 0.8
win_s = 4096 # fft size
hop_s = buffer_size // 2 # hop size
onset = aubio.onset("default", win_s, hop_s, RATE)

q = queue.Queue()

In [ ]:
def draw_pygame():
    running = True
    while running:
        key = pygame.key.get_pressed()

        if key[pygame.K_q]:
            running = False
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        if not q.empty():
            b = q.get()
            newCircle = Circle(random.randint(0, screenWidth), random.randint(0, screenHeight),
                               random.choice(colors), 700)
            circleList.append(newCircle)

        screen.fill(black)
        for place, circle in enumerate(circleList):
            if circle.size < 1:
                circleList.pop(place)
            else:
                pygame.draw.circle(screen, circle.color, (circle.x, circle.y), circle.size)
            circle.shrink()
        
        pygame.display.flip()
        clock.tick(90)

In [ ]:
def get_onsets():
    while True:
        try:
            buffer_size = 2048 # needed to change this to get undistorted audio
            audiobuffer = stream.read(buffer_size, exception_on_overflow=False)
            signal = np.fromstring(audiobuffer, dtype=np.float32)


            if onset(signal):
                q.put(True)

        except KeyboardInterrupt:
            print("*** Ctrl+C pressed, exiting")
            break


t = Thread(target=get_onsets, args=())
t.daemon = True
t.start()

draw_pygame()
get_onsets()

In [ ]:
stream.stop_stream()
stream.close()
pygame.display.quit()